In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

import sys
sys.path

import condacolab
condacolab.check()

!mamba install -c conda-forge rdkit
!pip install mordred

✨🍰✨ Everything looks OK!
✨🍰✨ Everything looks OK!

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit']

conda-forge/linux-64                   

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import os
import random
import keras

from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.models import load_model

from rdkit import Chem
from rdkit.Chem import PandasTools, AllChem, Draw, ChemicalFeatures
from rdkit.Chem import Descriptors
from rdkit import RDConfig
from rdkit import DataStructs

from mordred import Calculator, descriptors
from tqdm.auto import tqdm
from itertools import chain
from keras.models import load_model

In [ ]:
def seed_everything(seed):
    # Python의 난수 생성기 시드 고정
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

seed =42
seed_everything(seed)

# Define Featurizers

In [ ]:
#1 RDkit에서 제공하는 Descriptor
def getMolDescriptors(mol, missingVal=None):
    res = {}
    for nm,fn in Descriptors._descList:
        try:
            val = fn(mol)
        except:
            import traceback
            traceback.print_exc()
            val = missingVal
        res[nm] = val
    return res


#2 atomic environment/ 각 원자의 주변원자, 혼성오비탈, ring에 포함되어 있는 원자인지 / 각 atomic environment를 피쳐로 개수 count
def extract_fragments(smiles, hybridization_type=None, ring=False):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        hybridization = atom.GetHybridization()
        if (hybridization_type is None or hybridization == hybridization_type) and ring and atom.IsInRing():
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

        if (hybridization_type is None or hybridization == hybridization_type) and not ring and not atom.IsInRing():
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

def count_fragments(full, column_prefix, hybridization_type=None, ring=False):
    fragment_counts = {}
    for index, row in full.iterrows():
        smiles = row['SMILES']
        fragments = extract_fragments(smiles, hybridization_type, ring)

        fragment_count = {}
        for idx, fragment in enumerate(fragments):
            fragment_smiles = Chem.MolToSmiles(fragment)
            if fragment_smiles in fragment_count:
                fragment_count[fragment_smiles] += 1
            else:
                fragment_count[fragment_smiles] = 1

        fragment_counts[index] = fragment_count

    result_data = []
    for index, counts in fragment_counts.items():
        counts['SMILES'] = full.loc[index, 'SMILES']
        result_data.append(counts)

    result_df = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
    result_df.columns = [f'{column_prefix}_{column}' for column in result_df.columns]

    return result_df

def extract_sp3_carbon_fragments(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP3:
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

def count_sp3_fragments(full):
    fragment_counts = {}
    for index, row in full.iterrows():
        smiles = row['SMILES']
        fragments = extract_sp3_carbon_fragments(smiles)

        fragment_count = {}
        for idx, fragment in enumerate(fragments):
            fragment_smiles = Chem.MolToSmiles(fragment)
            if fragment_smiles in fragment_count:
                fragment_count[fragment_smiles] += 1
            else:
                fragment_count[fragment_smiles] = 1

        fragment_counts[index] = fragment_count

    result_data = []
    for index, counts in fragment_counts.items():
        counts['SMILES'] = full.loc[index, 'SMILES']
        result_data.append(counts)

    result_df = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
    result_df.columns = [f'sp3_{column}' for column in result_df.columns]

    return result_df

def count_NandO(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    n_count = sum(1 for atom in mol.GetAtoms() if atom.GetSymbol() == 'N')
    o_count = sum(1 for atom in mol.GetAtoms() if atom.GetSymbol() == 'O')
    return n_count + o_count

def count_3_and_4_membered_rings(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None

    in34 = sum(1 for ring in mol.GetRingInfo().AtomRings() if len(ring) in [3, 4])
    in56 = sum(1 for ring in mol.GetRingInfo().AtomRings() if len(ring) in [5, 6])

    return in34, in56

def count_non_conjugated_atoms(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    noncon_count = 0
    for atom in mol.GetAtoms():
        if not atom.IsInRing() and not atom.GetIsAromatic():
            noncon_count += 1

    return noncon_count

def calculate_stereocenters(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    chiral_centers = Chem.FindMolChiralCenters(mol, includeUnassigned=True)
    return len(chiral_centers)

def calculate_aromatic_proportion(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    aromatic_atoms = [atom for atom in mol.GetAtoms() if atom.GetIsAromatic()]
    total_atoms = mol.GetNumAtoms()
    if total_atoms == 0:
        return None
    return len(aromatic_atoms) / total_atoms

def calculate_heavy_atom_count(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return mol.GetNumHeavyAtoms()

def calculate_formal_charge(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return Chem.rdmolops.GetFormalCharge(mol)

def calculate_rule_of_five_satisfactions(smiles, mol_MW, accptHB, donorHB):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None

    QPlogPo_w = Descriptors.MolLogP(mol)

    satisfactions = 0
    if mol_MW >= 500:
        satisfactions += 1
    if QPlogPo_w >= 5:
        satisfactions += 1
    if donorHB > 5:
        satisfactions += 1
    if accptHB > 10:
        satisfactions += 1

    return satisfactions, QPlogPo_w

def calculate_molar_refractivity(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return Descriptors.MolMR(mol)
    return None

def calculate_vdw_volume(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return Descriptors.HeavyAtomCount(mol)
    return None

def calculate_molecular_flexibility(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return Descriptors.NumRotatableBonds(mol)
    return None

def calculate_electronegativity(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        total_eneg = sum([atom.GetAtomicNum() for atom in mol.GetAtoms()])
        return total_eneg
    return None

def calculate_pharmacophore_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return len([atom for atom in mol.GetAtoms() if atom.GetIsAromatic()])
    return None

def Custom_descriptors(full):
    NandO_data = []
    ring_data = []
    noncon_data = []
    stereocenters_data = []
    aromatic_proportion_data = []
    heavy_atom_data = []
    formal_charge_data = []
    rule_of_five_data = []
    data = []

    for index, row in full.iterrows():
        smiles = row['SMILES']
        nando_count = count_NandO(smiles)
        in34_count, in56_count = count_3_and_4_membered_rings(smiles)
        noncon_count = count_non_conjugated_atoms(smiles)
        stereocenters = calculate_stereocenters(smiles)
        proportion = calculate_aromatic_proportion(smiles)
        heavy_atom_count = calculate_heavy_atom_count(smiles)
        charge = calculate_formal_charge(smiles)
        mol_MW = row['Molecular_Weight']
        accptHB = row['Num_H_Acceptors']
        donorHB = row['Num_H_Donors']
        mr = calculate_molar_refractivity(smiles)
        vdw = calculate_vdw_volume(smiles)
        flex = calculate_molecular_flexibility(smiles)
        eneg = calculate_electronegativity(smiles)
        pharm = calculate_pharmacophore_features(smiles)


        rule_of_five_satisfactions, QPlogPo_w_value = calculate_rule_of_five_satisfactions(smiles, mol_MW, accptHB, donorHB)

        if nando_count is not None:
            NandO_data.append({"NandO": nando_count})
        if in34_count is not None and in56_count is not None:
            ring_data.append({"in34": in34_count, "in56": in56_count})
        if noncon_count is not None:
            noncon_data.append({"noncon": noncon_count})
        if stereocenters is not None:
            stereocenters_data.append({"stereocenters": stereocenters})
        if proportion is not None:
            aromatic_proportion_data.append({"aromatic_proportion": proportion})
        if heavy_atom_count is not None:
            heavy_atom_data.append({"heavy_atom_count": heavy_atom_count})
        if charge is not None:
            formal_charge_data.append({"formal_charge": charge})
        if rule_of_five_satisfactions is not None:
            rule_of_five_data.append({
                "rule_of_five_satisfactions": rule_of_five_satisfactions,
                "QPlogPo_w": QPlogPo_w_value
            })
        data.append({"Molar Refractivity": mr,
                  "VdW Volume": vdw,
                  "Flexibility": flex,
                  "Electronegativity": eneg,
                  "Pharmacophore Features": pharm})

    NandO_df = pd.DataFrame(NandO_data)
    ring_df = pd.DataFrame(ring_data)
    noncon_df = pd.DataFrame(noncon_data)
    stereocenters_df = pd.DataFrame(stereocenters_data)
    aromatic_proportion_df = pd.DataFrame(aromatic_proportion_data)
    heavy_atom_df = pd.DataFrame(heavy_atom_data)
    formal_charge_df = pd.DataFrame(formal_charge_data)
    rule_of_five_df = pd.DataFrame(rule_of_five_data)
    MVMEP_df = pd.DataFrame(data)

    property_df = pd.concat([NandO_df, ring_df, noncon_df, stereocenters_df,
                           aromatic_proportion_df, heavy_atom_df, formal_charge_df,
                           rule_of_five_df, MVMEP_df], axis=1)

    return property_df

# Preprocessing

In [ ]:

def process_duplicate_smiles(train):
    # 중복된 'SMILES' 값들을 찾고 중복된 행을 평균값으로 처리
    duplicates = train[train.duplicated(subset='SMILES', keep=False)].sort_values(by='SMILES').reset_index(drop=True)
    mean_mlm = duplicates.groupby('SMILES')['MLM'].mean().reset_index()
    mean_hlm = duplicates.groupby('SMILES')['HLM'].mean().reset_index()

    for index, row in mean_mlm.iterrows():
        smiles = row['SMILES']
        mlm_mean = row['MLM']
        hlm_mean = mean_hlm[mean_hlm['SMILES'] == smiles]['HLM'].values[0]
        duplicates.loc[duplicates['SMILES'] == smiles, 'MLM'] = mlm_mean
        duplicates.loc[duplicates['SMILES'] == smiles, 'HLM'] = hlm_mean
    mean_duplicate = duplicates.drop_duplicates().reset_index(drop=True)

    # 원래 데이터프레임(train)에서 중복된 'SMILES' 값을 업데이트
    for index, row in mean_duplicate.iterrows():
        s_value = row['SMILES']
        matching_indices = train[train['SMILES'] == s_value].index
        for train_index in matching_indices:
            train.at[train_index, 'MLM'] = row['MLM']
            train.at[train_index, 'HLM'] = row['HLM']

    # 중복 제거 후 결과 반환
    train = train.drop_duplicates(subset=['SMILES', 'HLM', 'MLM'], keep='first').reset_index(drop=True)

    return train


In [ ]:
# train_set

train = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train.csv")

# AlogP 결측제거
train.loc[2796,'AlogP'] =  round(Chem.Crippen.MolLogP(Chem.MolFromSmiles(train.loc[2796,'SMILES'])),3)
train.loc[3387,'AlogP'] =  round(Chem.Crippen.MolLogP(Chem.MolFromSmiles(train.loc[3387,'SMILES'])),3)

# 이상치 제거
train = train[train['LogD'] > - 4]

# SMILES 중복치 중, labels(MLM, HLM) 값 차이가 많이 나는 값 먼저 제거 # MLM, HLM이 큰 자료 선택
data_to_delete = ['TRAIN_1212', 'TRAIN_2332', 'TRAIN_2651']
train = train[~train['id'].isin(data_to_delete)]

# smiles 중복 제거, labels(MLM, HLM)은 mean으로 처리
train = process_duplicate_smiles(train)

# 외부데이터로 ChEMBL Database 활용

MLM_addon = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/chembl_liver_MLM_hanri_ver2.csv")
MLM_addon = MLM_addon[MLM_addon['Time_min']==30].dropna(subset=['LogD']).drop(columns=['Time_min', 'Concentration_uM'])
MLM_addon['HLM'] = np.NaN
MLM_addon = process_duplicate_smiles(MLM_addon)

HLM_addon = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/chembl_liver_HLM_hanri_ver2.csv")
HLM_addon = HLM_addon[HLM_addon['Time_min']==30].dropna(subset=['LogD']).drop(columns=['Computed_AlogP','Time_min', 'Concentration_uM'])
HLM_addon['MLM'] = np.NaN
HLM_addon = process_duplicate_smiles(HLM_addon)

print('중복 제거 후 train data 수: ',len(train))
print('ChEMBL MLM data 수: ',len(MLM_addon))
print('ChEMBL HLM data 수: ',len(HLM_addon))

full_MLM = pd.concat([train, MLM_addon], axis=0).reset_index(drop=True)
full_HLM = pd.concat([train, HLM_addon], axis=0).reset_index(drop=True)

print('전체 train MLM data 수: ',len(full_MLM))
print('전체 train HLM data 수: ',len(full_HLM))

full_MLM['Molecule'] = full_MLM['SMILES'].apply(Chem.MolFromSmiles)
full_HLM['Molecule'] = full_HLM['SMILES'].apply(Chem.MolFromSmiles)

중복 제거 후 train data 수:  3469
ChEMBL MLM data 수:  1501
ChEMBL HLM data 수:  1755
전체 train MLM data 수:  4970
전체 train HLM data 수:  5224


# feauterize_MLM_data

In [ ]:
#1 Atomic environment

result_df_sp3 = count_sp3_fragments(full_MLM)
result_df_sp2 = count_fragments(full_MLM, 'sp2', Chem.HybridizationType.SP2)
result_df_ring_sp2 = count_fragments(full_MLM, 'ring_sp2', Chem.HybridizationType.SP2, ring=True)
result_df_sp = count_fragments(full_MLM, 'sp', Chem.HybridizationType.SP)

atomic_env_MLM_df = pd.concat([result_df_sp3, result_df_sp2, result_df_ring_sp2, result_df_sp], axis =1)

#2 Custom Descriptors

custom_MLM_df = Custom_descriptors(full_MLM)

#3 RDkit Descriptors

allDescrs = [getMolDescriptors(m) for m in tqdm(full_MLM['Molecule'], desc="Generating descriptors")]
full_MLM_Descrs = pd.DataFrame(allDescrs)

#4 mordred Descriptors

calc = Calculator(descriptors, ignore_3D=True)
mord_desc_MLM_df = calc.pandas([Chem.MolFromSmiles(x) for x in full_MLM.SMILES])
mord_desc_MLM_df.columns = ['mord_'+ column for column in mord_desc_MLM_df.columns]
mord_desc_MLM_df.drop(columns=mord_desc_MLM_df.select_dtypes(include=['object']).columns, inplace=True) # object 형식 열 삭제
bool_columns = mord_desc_MLM_df.select_dtypes(include=['bool']).columns
mord_desc_MLM_df[bool_columns] = mord_desc_MLM_df[bool_columns].astype(int) # bool 형식 열을 0과 1로 인코딩

# Concatenate

train_MLM = pd.concat([full_MLM, full_MLM_Descrs, mord_desc_MLM_df, atomic_env_MLM_df, custom_MLM_df], axis=1).drop(columns=['Molecule', 'MolWt', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'])

#5 beyond_Lipinski feature 추가

train_MLM['beyond_Lipinski'] = 0

train_MLM.loc[train_MLM['Molecular_Weight'] >= 500, 'beyond_Lipinski'] += 1
train_MLM.loc[train_MLM['LogD'] >= 5, 'beyond_Lipinski'] += 1
train_MLM.loc[train_MLM['Num_H_Donors'] >= 5, 'beyond_Lipinski'] += 1
train_MLM.loc[train_MLM['Num_H_Acceptors'] >= 10, 'beyond_Lipinski'] += 1
train_MLM.loc[train_MLM['Num_RotatableBonds'] >= 10, 'beyond_Lipinski'] += 1
train_MLM.loc[(train_MLM['Num_H_Donors'] + train_MLM['Num_H_Acceptors']) >= 12, 'beyond_Lipinski'] += 1
train_MLM.loc[train_MLM['TPSA'] >= 140, 'beyond_Lipinski'] += 1
train_MLM.loc[train_MLM['FractionCSP3'] <= 0.3, 'beyond_Lipinski'] += 1
train_MLM.loc[train_MLM['NumAromaticRings'] >= 5, 'beyond_Lipinski'] += 1
train_MLM.loc[train_MLM['NandO'] >= 10, 'beyond_Lipinski'] += 1
train_MLM.loc[train_MLM['NHOHCount'] >= 5, 'beyond_Lipinski'] += 1

# Molecular_PolarSurfaceArea 결측 처리

train_MLM["Molecular_PolarSurfaceArea"] = np.where(pd.isna(train_MLM["Molecular_PolarSurfaceArea"]), train_MLM["TPSA"], train_MLM["Molecular_PolarSurfaceArea"])

Generating descriptors:   0%|          | 0/4970 [00:00<?, ?it/s]

  5%|▍         | 248/4970 [01:14<1:02:03,  1.27it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 84%|████████▍ | 4185/4970 [20:09<08:51,  1.48it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


...

 85%|████████▌ | 4227/4970 [20:28<07:04,  1.75it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 85%|████████▌ | 4228/4970 [20:29<07:52,  1.57it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 4970/4970 [24:35<00:00,  3.37it/s]


In [ ]:
#
train_MLM.to_csv('/content/drive/MyDrive/Seed_learning_sub/train_MLM.csv', index=False)
# train_MLM = pd.read_csv('/content/drive/MyDrive/Seed_learning_sub/train_MLM.csv')

# 숫자형식 컬럼들의 min-max 정규화
scaler_MLM = MinMaxScaler()
non_features = ['id', 'SMILES', 'MLM', 'HLM','Fingerprint']
features_MLM = [column for column in train_MLM.columns if column not in non_features]
train_MLM[features_MLM] = scaler_MLM.fit_transform(train_MLM[features_MLM])

#
train_MLM = train_MLM.drop(columns=['HLM']).dropna(axis=0)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasatt

In [ ]:
train_MLM

,id,SMILES,MLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,...,heavy_atom_count,formal_charge,rule_of_five_satisfactions,QPlogPo_w,Molar Refractivity,VdW Volume,Flexibility,Electronegativity,Pharmacophore Features,beyond_Lipinski
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,0.551836,0.222290,0.333333,0.2,0.210526,0.529136,0.389270,...,0.218391,0.666667,0.0,0.480778,0.247429,0.218391,0.275862,0.223776,0.410256,0.0
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,0.469473,0.142015,0.133333,0.1,0.052632,0.442839,0.239538,...,0.137931,0.666667,0.0,0.437703,0.177609,0.137931,0.068966,0.141608,0.282051,0.0
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,0.425948,0.138735,0.333333,0.0,0.078947,0.396237,0.201951,...,0.149425,0.666667,0.0,0.260507,0.171813,0.149425,0.103448,0.138112,0.333333,0.0
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,0.666087,0.298570,0.400000,0.0,0.131579,0.546285,0.304785,...,0.298851,0.666667,0.0,0.482106,0.331812,0.298851,0.172414,0.297203,0.435897,0.0
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,0.482016,0.115202,0.200000,0.0,0.026316,0.455938,0.133668,...,0.126437,0.666667,0.0,0.393449,0.147193,0.126437,0.034483,0.115385,0.307692,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4965,CHEMBL4444111,Cc1nc(NC(C)(C)c2cc3cc(Cl)ccc3[nH]c2=O)ccc1C#N,67.000,0.615385,0.183675,0.266667,0.2,0.078947,0.535567,0.267165,...,0.183908,0.666667,0.0,0.499431,0.218544,0.183908,0.103448,0.187063,0.410256,0.1
4966,CHEMBL4542800,Cc1cc(NCc2cc3cc(Cl)ccc3[nH]c2=O)ccc1C#N,49.000,0.605561,0.160140,0.200000,0.2,0.078947,0.560178,0.223200,...,0.160920,0.666667,0.0,0.488710,0.196207,0.160920,0.103448,0.164336,0.410256,0.1
4967,CHEMBL4437186,COc1cc(NCc2cc3cc(Cl)c(OCCN4CCOCC4)cc3[nH]c2=O)...,13.100,0.560979,0.277740,0.466667,0.2,0.210526,0.477612,0.328695,...,0.275862,0.666667,0.0,0.440187,0.304703,0.275862,0.275862,0.281469,0.410256,0.0
4968,CHEMBL4451408,COc1nc(N[C@@H](C)c2cc3cc(Cl)ccc3[nH]c2=O)ncc1C#N,100.000,0.533777,0.186073,0.400000,0.2,0.105263,0.488727,0.342611,...,0.183908,0.666667,0.0,0.410520,0.202848,0.183908,0.137931,0.192308,0.410256,0.1


In [ ]:
#1 Atomic environment

result_df_sp3 = count_sp3_fragments(full_HLM)
result_df_sp2 = count_fragments(full_HLM, 'sp2', Chem.HybridizationType.SP2)
result_df_ring_sp2 = count_fragments(full_HLM, 'ring_sp2', Chem.HybridizationType.SP2, ring=True)
result_df_sp = count_fragments(full_HLM, 'sp', Chem.HybridizationType.SP)

atomic_env_HLM_df = pd.concat([result_df_sp3, result_df_sp2, result_df_ring_sp2, result_df_sp], axis =1)

#2 Custom Descriptors

custom_HLM_df = Custom_descriptors(full_HLM)

#3 RDkit Descriptors

allDescrs = [getMolDescriptors(m) for m in tqdm(full_HLM['Molecule'], desc="Generating descriptors")]
full_HLM_Descrs = pd.DataFrame(allDescrs)

#4 mordred Descriptors

calc = Calculator(descriptors, ignore_3D=True)
mord_desc_HLM_df = calc.pandas([Chem.MolFromSmiles(x) for x in full_HLM.SMILES])
mord_desc_HLM_df.columns = ['mord_'+ column for column in mord_desc_HLM_df.columns]
mord_desc_HLM_df.drop(columns=mord_desc_HLM_df.select_dtypes(include=['object']).columns, inplace=True) # object 형식 열 삭제
bool_columns = mord_desc_HLM_df.select_dtypes(include=['bool']).columns
mord_desc_HLM_df[bool_columns] = mord_desc_HLM_df[bool_columns].astype(int) # bool 형식 열을 0과 1로 인코딩

# Concatenate

train_HLM = pd.concat([full_HLM, full_HLM_Descrs, mord_desc_HLM_df, atomic_env_HLM_df, custom_HLM_df], axis=1).drop(columns=['Molecule', 'MolWt', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'])

# beyond_Lipinski feature 추가

train_HLM['beyond_Lipinski'] = 0

train_HLM.loc[train_HLM['Molecular_Weight'] >= 500, 'beyond_Lipinski'] += 1
train_HLM.loc[train_HLM['LogD'] >= 5, 'beyond_Lipinski'] += 1
train_HLM.loc[train_HLM['Num_H_Donors'] >= 5, 'beyond_Lipinski'] += 1
train_HLM.loc[train_HLM['Num_H_Acceptors'] >= 10, 'beyond_Lipinski'] += 1
train_HLM.loc[train_HLM['Num_RotatableBonds'] >= 10, 'beyond_Lipinski'] += 1
train_HLM.loc[(train_HLM['Num_H_Donors'] + train_HLM['Num_H_Acceptors']) >= 12, 'beyond_Lipinski'] += 1
train_HLM.loc[train_HLM['TPSA'] >= 140, 'beyond_Lipinski'] += 1
train_HLM.loc[train_HLM['FractionCSP3'] <= 0.3, 'beyond_Lipinski'] += 1
train_HLM.loc[train_HLM['NumAromaticRings'] >= 5, 'beyond_Lipinski'] += 1
train_HLM.loc[train_HLM['NandO'] >= 10, 'beyond_Lipinski'] += 1
train_HLM.loc[train_HLM['NHOHCount'] >= 5, 'beyond_Lipinski'] += 1

# Molecular_PolarSurfaceArea 결측 처리

train_HLM["Molecular_PolarSurfaceArea"] = np.where(pd.isna(train_HLM["Molecular_PolarSurfaceArea"]), train_HLM["TPSA"], train_HLM["Molecular_PolarSurfaceArea"])

Generating descriptors:   0%|          | 0/5224 [00:00<?, ?it/s]

  5%|▍         | 248/5224 [01:12<1:08:43,  1.21it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 22%|██▏       | 1171/5224 [05:23<45:13,  1.49it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


...

 86%|████████▌ | 4479/5224 [21:38<09:52,  1.26it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 89%|████████▉ | 4637/5224 [22:32<06:58,  1.40it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 5224/5224 [25:58<00:00,  3.35it/s]


In [ ]:
train_HLM.to_csv('/content/drive/MyDrive/Seed_learning_sub/train_HLM.csv', index=False)
# train_HLM = pd.read_csv('/content/drive/MyDrive/Seed_learning_sub/train_HLM.csv')

# 숫자형식 컬럼들의 min-max 정규화
scaler_HLM = MinMaxScaler()
non_features = ['id', 'SMILES', 'MLM', 'HLM','Fingerprint']
features_HLM = [column for column in train_HLM.columns if column not in non_features]
train_HLM[features_HLM] = scaler_HLM.fit_transform(train_HLM[features_HLM])

#
train_HLM = train_HLM.drop(columns=['MLM']).dropna(axis=0)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasatt

In [ ]:
train_HLM

,id,SMILES,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,...,heavy_atom_count,formal_charge,rule_of_five_satisfactions,QPlogPo_w,Molar Refractivity,VdW Volume,Flexibility,Electronegativity,Pharmacophore Features,beyond_Lipinski
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,50.680,0.551836,0.192138,0.3125,0.153846,0.210526,0.576357,0.320878,...,0.209302,0.666667,0.0,0.480778,0.218769,0.209302,0.242424,0.192727,0.410256,0.0
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,50.590,0.469473,0.108751,0.1250,0.076923,0.052632,0.498714,0.197453,...,0.127907,0.666667,0.0,0.437703,0.146289,0.127907,0.060606,0.107273,0.282051,0.0
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,80.892,0.425948,0.105343,0.3125,0.000000,0.078947,0.456786,0.166470,...,0.139535,0.666667,0.0,0.260507,0.140273,0.139535,0.090909,0.103636,0.333333,0.0
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,2.000,0.666087,0.271376,0.3750,0.000000,0.131579,0.591786,0.251237,...,0.290698,0.666667,0.0,0.482106,0.306365,0.290698,0.151515,0.269091,0.435897,0.0
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,99.990,0.482016,0.080898,0.1875,0.000000,0.026316,0.510500,0.110183,...,0.116279,0.666667,0.0,0.393449,0.114715,0.116279,0.030303,0.080000,0.307692,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5219,CHEMBL3736307,Cc1nc(C(=O)N(C)C)c(N)c2c1C(=O)N(Cc1ccccc1)C2=O,72.000,0.416654,0.139857,0.3125,0.076923,0.078947,0.465714,0.262483,...,0.174419,0.666667,0.0,0.282404,0.163154,0.174419,0.090909,0.141818,0.307692,0.1
5220,CHEMBL3736387,Cc1nc(C(=O)NC#N)c(N)c2c1C(=O)N(Cc1ccccc1)C2=O,99.000,0.379628,0.137290,0.3750,0.153846,0.078947,0.448571,0.354082,...,0.174419,0.666667,0.0,0.241949,0.147064,0.174419,0.090909,0.143636,0.307692,0.1
5221,CHEMBL3769968,C=CCOc1cc(F)c(F)cc1C1CC1CN.Cl,91.900,0.501285,0.087142,0.1250,0.076923,0.131579,0.338571,0.089997,...,0.093023,0.666667,0.0,0.409591,0.088709,0.093023,0.151515,0.083636,0.153846,0.0
5222,CHEMBL3798025,O=C(c1cc(-c2cccnc2)[nH]n1)N1CCN(C(c2ccccc2)c2c...,63.000,0.609340,0.211515,0.2500,0.076923,0.131579,0.595714,0.173977,...,0.255814,0.666667,0.0,0.492451,0.267321,0.255814,0.151515,0.212727,0.589744,0.1


#학습(MLP, Seed ensemble)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

seeds = [ 1, 2, 3, 4, 5 ]

for seed in seeds:
  seed_everything(seed)

  # Define features and targets
  non_features = ['id', 'SMILES', 'MLM', 'HLM', 'Fingerprint']
  features = [column for column in train_MLM.columns if column not in non_features]
  mlm_target = "MLM"
  hlm_target = "HLM"

  # Initialize KFold
  n_splits = 10
  kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)

  # Initialize arrays to store models and scores
  reg_mlms = []
  reg_hlms = []

  # Initialize arrays to store RMSE scores
  mlm_rmse_scores = []
  hlm_rmse_scores = []

  # Loop through KFold splits
  for i, (train_index, valid_index) in enumerate(kf.split(train_MLM)):
      df_train = train_MLM.iloc[train_index]
      df_valid = train_MLM.iloc[valid_index]

      x_train_num = df_train[features].values

      y_mlm_train = df_train[mlm_target].values

      x_valid_num = df_valid[features].values

      y_mlm_valid = df_valid[mlm_target].values


      x1_input = keras.Input(shape=(x_train_num.shape[1],))
      x1 = layers.Dense(256, activation='relu')(x1_input)
      x1 = layers.BatchNormalization()(x1)
      x1 = layers.Dropout(0.2)(x1)
      x1 = layers.Dense(1024, activation='relu')(x1)
      x1 = layers.BatchNormalization()(x1)
      x1 = layers.Dropout(0.2)(x1)
      x1 = layers.Dense(512, activation='relu')(x1)
      x1 = layers.BatchNormalization()(x1)
      x1 = layers.Dropout(0.2)(x1)
      x1 = layers.Dense(256, activation='relu')(x1)
      x1 = layers.BatchNormalization()(x1)
      x1 = layers.Dropout(0.2)(x1)
      x1 = layers.Dense(128, activation='relu')(x1)
      x1 = layers.BatchNormalization()(x1)

      outputs = layers.Dense(1)(x1)

      model = keras.Model(inputs=x1_input, outputs=outputs)

      lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate=0.001,
      decay_steps=8,
      decay_rate=0.98)
      optimizer_decay = keras.optimizers.SGD(learning_rate=lr_schedule)

      model.compile(optimizer=optimizer_decay, loss='mean_squared_error', metrics=[RootMeanSquaredError()])


      # Train the model
      checkpoint_mlm = ModelCheckpoint(f'/content/drive/MyDrive/Seed_learning_sub/{seed}_model_mlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
      early_stopping_mlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

      model.fit(x_train_num, y_mlm_train, epochs=500, batch_size=16, verbose=1, validation_data=(x_valid_num, y_mlm_valid),
                callbacks=[checkpoint_mlm, early_stopping_mlm])

      reg_mlms.append(model)

      # Calculate RMSE for MLM predictions
      y_mlm_pred = model.predict(x_valid_num)
      mlm_rmse = math.sqrt(mean_squared_error(y_mlm_valid, y_mlm_pred))
      print(mlm_rmse)
      mlm_rmse_scores.append(mlm_rmse)

      # Define features and targets
  non_features = ['id', 'SMILES', 'MLM', 'HLM', 'Fingerprint']
  features = [column for column in train_HLM.columns if column not in non_features]
  mlm_target = "MLM"
  hlm_target = "HLM"

  # Initialize KFold
  n_splits = 10
  kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)

  # Initialize arrays to store models and scores
  reg_mlms = []
  reg_hlms = []

  # Initialize arrays to store RMSE scores
  mlm_rmse_scores = []
  hlm_rmse_scores = []

  # Loop through KFold splits
  for i, (train_index, valid_index) in enumerate(kf.split(train_HLM)):
      df_train = train_HLM.iloc[train_index]
      df_valid = train_HLM.iloc[valid_index]

      x_train_num = df_train[features].values

      y_hlm_train = df_train[hlm_target].values

      x_valid_num = df_valid[features].values

      y_hlm_valid = df_valid[hlm_target].values


      x1_input = keras.Input(shape=(x_train_num.shape[1],))
      x1 = layers.Dense(256, activation='relu')(x1_input)
      x1 = layers.BatchNormalization()(x1)
      x1 = layers.Dropout(0.2)(x1)
      x1 = layers.Dense(1024, activation='relu')(x1)
      x1 = layers.BatchNormalization()(x1)
      x1 = layers.Dropout(0.2)(x1)
      x1 = layers.Dense(512, activation='relu')(x1)
      x1 = layers.BatchNormalization()(x1)
      x1 = layers.Dropout(0.2)(x1)
      x1 = layers.Dense(256, activation='relu')(x1)
      x1 = layers.BatchNormalization()(x1)
      x1 = layers.Dropout(0.2)(x1)
      x1 = layers.Dense(128, activation='relu')(x1)
      x1 = layers.BatchNormalization()(x1)

      outputs = layers.Dense(1)(x1)

      hmodel = keras.Model(inputs=x1_input, outputs=outputs)

      lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate=0.001,
      decay_steps=8,
      decay_rate=0.98)
      optimizer_decay = keras.optimizers.SGD(learning_rate=lr_schedule)

      hmodel.compile(optimizer=optimizer_decay, loss='mean_squared_error', metrics=[RootMeanSquaredError()])


      # Train the model
      checkpoint_hlm = ModelCheckpoint(f'/content/drive/MyDrive/Seed_learning_sub/{seed}_model_hlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
      early_stopping_hlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

      hmodel.fit(x_train_num, y_hlm_train, epochs=500, batch_size=16, verbose=1, validation_data=(x_valid_num, y_hlm_valid),
                callbacks=[checkpoint_hlm, early_stopping_hlm])

      reg_hlms.append(model)

      # Calculate RMSE for MLM predictions
      y_hlm_pred = hmodel.predict(x_valid_num)
      hlm_rmse = math.sqrt(mean_squared_error(y_hlm_valid, y_hlm_pred))
      print(hlm_rmse)
      hlm_rmse_scores.append(hlm_rmse)


Epoch 1/500
278/279 [============================>.] - ETA: 0s - loss: 1193.6835 - root_mean_squared_error: 34.5497

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


279/279 [==============================] - 10s 10ms/step - loss: 1194.3876 - root_mean_squared_error: 34.5599 - val_loss: 1245.6158 - val_root_mean_squared_error: 35.2933
Epoch 2/500
279/279 [==============================] - 2s 7ms/step - loss: 937.7946 - root_mean_squared_error: 30.6234 - val_loss: 938.9605 - val_root_mean_squared_error: 30.6425
Epoch 3/500
279/279 [==============================] - 2s 7ms/step - loss: 868.4060 - root_mean_squared_error: 29.4687 - val_loss: 881.6891 - val_root_mean_squared_error: 29.6932
Epoch 4/500
279/279 [==============================] - 2s 7ms/step - loss: 832.0773 - root_mean_squared_error: 28.8457 - val_loss: 865.4286 - val_root_mean_squared_error: 29.4182
Epoch 5/500
279/279 [==============================] - 2s 7ms/step - loss: 822.8331 - root_mean_squared_error: 28.6851 - val_loss: 861.1990 - val_root_mean_squared_error: 29.3462
Epoch 6/500
279/279 [==============================] - 2s 6ms/step - loss: 801.5679 - root_mean_squared_error: 28

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


279/279 [==============================] - 5s 10ms/step - loss: 1226.1323 - root_mean_squared_error: 35.0162 - val_loss: 938.4134 - val_root_mean_squared_error: 30.6335
Epoch 2/500
279/279 [==============================] - 2s 6ms/step - loss: 921.2932 - root_mean_squared_error: 30.3528 - val_loss: 899.7326 - val_root_mean_squared_error: 29.9955
Epoch 3/500
279/279 [==============================] - 2s 6ms/step - loss: 872.5394 - root_mean_squared_error: 29.5388 - val_loss: 874.1743 - val_root_mean_squared_error: 29.5664
Epoch 4/500
279/279 [==============================] - 2s 6ms/step - loss: 834.3990 - root_mean_squared_error: 28.8860 - val_loss: 847.9565 - val_root_mean_squared_error: 29.1197
Epoch 5/500
279/279 [==============================] - 2s 6ms/step - loss: 818.6202 - root_mean_squared_error: 28.6115 - val_loss: 836.8606 - val_root_mean_squared_error: 28.9285
Epoch 6/500
279/279 [==============================] - 2s 6ms/step - loss: 808.4368 - root_mean_squared_error: 28.4

...

294/294 [==============================] - 2s 7ms/step - loss: 941.3821 - root_mean_squared_error: 30.6819 - val_loss: 921.3459 - val_root_mean_squared_error: 30.3537
Epoch 3/500
294/294 [==============================] - 2s 7ms/step - loss: 886.4849 - root_mean_squared_error: 29.7739 - val_loss: 889.5051 - val_root_mean_squared_error: 29.8246
Epoch 4/500
294/294 [==============================] - 2s 6ms/step - loss: 857.1208 - root_mean_squared_error: 29.2766 - val_loss: 869.5472 - val_root_mean_squared_error: 29.4881
Epoch 5/500
294/294 [==============================] - 2s 6ms/step - loss: 834.5549 - root_mean_squared_error: 28.8887 - val_loss: 873.6771 - val_root_mean_squared_error: 29.5580
Epoch 6/500
294/294 [==============================] - 2s 6ms/step - loss: 829.8739 - root_mean_squared_error: 28.8075 - val_loss: 874.7684 - val_root_mean_squared_error: 29.5765
Epoch 7/500
294/294 [==============================] - 2s 6ms/step - loss: 823.4164 - root_mean_squared_error: 28.695

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


294/294 [==============================] - 2s 7ms/step - loss: 973.2508 - root_mean_squared_error: 31.1970 - val_loss: 965.9829 - val_root_mean_squared_error: 31.0803
Epoch 3/500
294/294 [==============================] - 2s 6ms/step - loss: 906.8044 - root_mean_squared_error: 30.1132 - val_loss: 1041.0980 - val_root_mean_squared_error: 32.2660
Epoch 4/500
294/294 [==============================] - 2s 7ms/step - loss: 864.5759 - root_mean_squared_error: 29.4037 - val_loss: 922.8188 - val_root_mean_squared_error: 30.3779
Epoch 5/500
294/294 [==============================] - 2s 6ms/step - loss: 842.0668 - root_mean_squared_error: 29.0184 - val_loss: 928.8934 - val_root_mean_squared_error: 30.4778
Epoch 6/500
294/294 [==============================] - 2s 7ms/step - loss: 838.3531 - root_mean_squared_error: 28.9543 - val_loss: 912.5594 - val_root_mean_squared_error: 30.2086
Epoch 7/500
294/294 [==============================] - 2s 6ms/step - loss: 837.1920 - root_mean_squared_error: 28.93

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


294/294 [==============================] - 2s 7ms/step - loss: 969.9440 - root_mean_squared_error: 31.1439 - val_loss: 940.6052 - val_root_mean_squared_error: 30.6693
Epoch 3/500
294/294 [==============================] - 2s 6ms/step - loss: 906.8623 - root_mean_squared_error: 30.1142 - val_loss: 927.7780 - val_root_mean_squared_error: 30.4594
Epoch 4/500
294/294 [==============================] - 2s 7ms/step - loss: 870.2336 - root_mean_squared_error: 29.4997 - val_loss: 898.5428 - val_root_mean_squared_error: 29.9757
Epoch 5/500
294/294 [==============================] - 2s 7ms/step - loss: 851.3920 - root_mean_squared_error: 29.1786 - val_loss: 891.6223 - val_root_mean_squared_error: 29.8600
Epoch 6/500
294/294 [==============================] - 2s 6ms/step - loss: 842.4283 - root_mean_squared_error: 29.0246 - val_loss: 884.6042 - val_root_mean_squared_error: 29.7423
Epoch 7/500
294/294 [==============================] - 2s 7ms/step - loss: 828.4584 - root_mean_squared_error: 28.783

# Prediction

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/test.csv")
test.loc[10,'AlogP'] =  round(Chem.Crippen.MolLogP(Chem.MolFromSmiles(test.loc[10,'SMILES'])),3)
test['Molecule'] = test['SMILES'].apply(Chem.MolFromSmiles)

In [ ]:
#1 Atomic environment

result_df_sp3 = count_sp3_fragments(test)
result_df_sp2 = count_fragments(test, 'sp2', Chem.HybridizationType.SP2)
result_df_ring_sp2 = count_fragments(test, 'ring_sp2', Chem.HybridizationType.SP2, ring=True)
result_df_sp = count_fragments(test, 'sp', Chem.HybridizationType.SP)

atomic_env_test_df = pd.concat([result_df_sp3, result_df_sp2, result_df_ring_sp2, result_df_sp], axis =1)

#2 Custom Descriptors

custom_test_df = Custom_descriptors(test)

#3 RDkit Descriptors

allDescrs = [getMolDescriptors(m) for m in tqdm(test['Molecule'], desc="Generating descriptors")]
test_Descrs = pd.DataFrame(allDescrs)

#4 mordred Descriptors

calc = Calculator(descriptors, ignore_3D=True)
mord_desc_test_df = calc.pandas([Chem.MolFromSmiles(x) for x in test.SMILES])
mord_desc_test_df.columns = ['mord_'+ column for column in mord_desc_test_df.columns]
mord_desc_test_df.drop(columns=mord_desc_test_df.select_dtypes(include=['object']).columns, inplace=True) # object 형식 열 삭제
bool_columns = mord_desc_test_df.select_dtypes(include=['bool']).columns
mord_desc_test_df[bool_columns] = mord_desc_test_df[bool_columns].astype(int) # bool 형식 열을 0과 1로 인코딩

# Concatenate

test_input = pd.concat([test, test_Descrs, mord_desc_test_df, atomic_env_test_df, custom_test_df], axis=1).drop(columns=['Molecule', 'MolWt', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'])

# beyond_Lipinski feature 추가

test_input['beyond_Lipinski'] = 0

test_input.loc[test_input['Molecular_Weight'] >= 500, 'beyond_Lipinski'] += 1
test_input.loc[test_input['LogD'] >= 5, 'beyond_Lipinski'] += 1
test_input.loc[test_input['Num_H_Donors'] >= 5, 'beyond_Lipinski'] += 1
test_input.loc[test_input['Num_H_Acceptors'] >= 10, 'beyond_Lipinski'] += 1
test_input.loc[test_input['Num_RotatableBonds'] >= 10, 'beyond_Lipinski'] += 1
test_input.loc[(test_input['Num_H_Donors'] + test_input['Num_H_Acceptors']) >= 12, 'beyond_Lipinski'] += 1
test_input.loc[test_input['TPSA'] >= 140, 'beyond_Lipinski'] += 1
test_input.loc[test_input['FractionCSP3'] <= 0.3, 'beyond_Lipinski'] += 1
test_input.loc[test_input['NumAromaticRings'] >= 5, 'beyond_Lipinski'] += 1
test_input.loc[test_input['NandO'] >= 10, 'beyond_Lipinski'] += 1
test_input.loc[test_input['NHOHCount'] >= 5, 'beyond_Lipinski'] += 1

# Molecular_PolarSurfaceArea 결측 처리

test_input["Molecular_PolarSurfaceArea"] = np.where(pd.isna(test_input["Molecular_PolarSurfaceArea"]), test_input["TPSA"], test_input["Molecular_PolarSurfaceArea"])

Generating descriptors:   0%|          | 0/483 [00:00<?, ?it/s]

100%|██████████| 483/483 [02:12<00:00,  3.64it/s]


In [ ]:
# train에 맞게 test input 맞춰주기

non_features = ['id', 'SMILES', 'MLM', 'HLM','Fingerprint']

features_MLM = [column for column in train_MLM.columns if column not in non_features]
features_HLM = [column for column in train_HLM.columns if column not in non_features]

train_only_col_MLM = [column for column in train_MLM.columns if column not in test_input.columns]
train_only_col_HLM = [column for column in train_HLM.columns if column not in test_input.columns]

empty_columns_MLM = pd.DataFrame(columns=train_only_col_MLM)
empty_columns_HLM = pd.DataFrame(columns=train_only_col_HLM)

test_input_MLM = pd.concat([test_input, empty_columns_MLM], axis=1)
test_input_HLM = pd.concat([test_input, empty_columns_HLM], axis=1)

test_input_MLM = test_input_MLM[train_MLM.columns].fillna(0)
test_input_HLM = test_input_HLM[train_HLM.columns].fillna(0)

test_input_MLM[features_MLM] = scaler_MLM.transform(test_input_MLM[features_MLM])
test_input_HLM[features_HLM] = scaler_HLM.transform(test_input_HLM[features_HLM])

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasatt

In [ ]:
test_input_MLM

,id,SMILES,MLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,...,heavy_atom_count,formal_charge,rule_of_five_satisfactions,QPlogPo_w,Molar Refractivity,VdW Volume,Flexibility,Electronegativity,Pharmacophore Features,beyond_Lipinski
0,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,0,0.505138,0.190703,0.266667,0.2,0.184211,0.479597,0.305331,...,0.183908,0.666667,0.0,0.361629,0.228923,0.183908,0.241379,0.187063,0.282051,0.0
1,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,0,0.349781,0.197908,0.333333,0.0,0.078947,0.316847,0.221938,...,0.206897,0.666667,0.0,0.311657,0.213670,0.206897,0.103448,0.199301,0.307692,0.0
2,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,0,0.628684,0.179287,0.266667,0.4,0.131579,0.610988,0.305672,...,0.195402,0.666667,0.0,0.430762,0.226115,0.195402,0.103448,0.180070,0.461538,0.1
3,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,0,0.441212,0.177621,0.333333,0.0,0.052632,0.412909,0.265937,...,0.195402,0.666667,0.0,0.329218,0.202293,0.195402,0.068966,0.183566,0.512821,0.1
4,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,0,0.397688,0.184151,0.266667,0.0,0.052632,0.283820,0.197517,...,0.195402,0.666667,0.0,0.266096,0.220983,0.195402,0.068966,0.183566,0.333333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,0,0.623470,0.146095,0.133333,0.1,0.184211,0.604398,0.176984,...,0.149425,0.666667,0.0,0.475929,0.180459,0.149425,0.241379,0.136364,0.128205,0.0
479,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,0,0.259634,0.169552,0.333333,0.0,0.026316,0.132582,0.228248,...,0.172414,0.666667,0.0,0.162467,0.178281,0.172414,0.034483,0.166084,0.000000,0.0
480,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,0,0.440985,0.180882,0.200000,0.1,0.078947,0.412671,0.226747,...,0.195402,0.666667,0.0,0.352927,0.208605,0.195402,0.103448,0.183566,0.307692,0.1
481,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,0,0.365271,0.174198,0.200000,0.2,0.052632,0.303985,0.226474,...,0.126437,0.666667,0.0,0.346236,0.141601,0.126437,0.068966,0.174825,0.307692,0.1


In [ ]:
seeds = [1, 2, 3, 4, 5]
n_splits = 10

# Initialize empty arrays to store predictions for MLM and HLM
mlm_predictions_all = []
hlm_predictions_all = []

for seed in seeds:
    seed_everything(seed)

    # Load MLM models
    mlm_models = [load_model(f'/content/drive/MyDrive/Seed_learning_sub/{seed}_model_mlm_fold{i}.h5') for i in range(n_splits)]

    # Load HLM models
    hlm_models = [load_model(f'/content/drive/MyDrive/Seed_learning_sub/{seed}_model_hlm_fold{i}.h5') for i in range(n_splits)]

    # Separate test features
    x_test_mlm = test_input_MLM.drop(columns=['id', 'SMILES', 'MLM']).values
    x_test_hlm = test_input_HLM.drop(columns=['id', 'SMILES', 'HLM']).values

    mlm_predictions = []
    hlm_predictions = []

    for model_mlm, model_hlm in zip(mlm_models, hlm_models):
        mlm_predictions.append(model_mlm.predict(x_test_mlm))
        hlm_predictions.append(model_hlm.predict(x_test_hlm))

    # Convert prediction lists to numpy arrays
    mlm_predictions = np.array(mlm_predictions)
    hlm_predictions = np.array(hlm_predictions)

    # Calculate ensemble predictions for MLM and HLM
    mlm_ensemble_prediction = mlm_predictions.mean(axis=0)
    hlm_ensemble_prediction = hlm_predictions.mean(axis=0)

    # Append ensemble predictions to the arrays
    mlm_predictions_all.append(mlm_ensemble_prediction)
    hlm_predictions_all.append(hlm_ensemble_prediction)

# Calculate final ensemble predictions by averaging across all seeds
final_mlm_prediction = np.mean(mlm_predictions_all, axis=0)
final_hlm_prediction = np.mean(hlm_predictions_all, axis=0)

# Load the sample submission dataframe
df_submission = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/sample_submission.csv")

# Assign the final MLM and HLM predictions to the dataframe
df_submission["MLM"] = final_mlm_prediction
df_submission["HLM"] = final_hlm_prediction

# Save the final submission file
df_submission.to_csv("/content/drive/MyDrive/Seed_learning_sub/MLP_seed_ensemble.csv", index=False, encoding="utf-8-sig")

16/16 [==============================] - 0s 2ms/step


In [ ]:
# seeds = [ 1, 2, 3, 4, 5 ]
# n_splits = 10

# for seed in seeds:
#   seed_everything(seed)

#   # Load MLM models
#   mlm_models = [load_model(f'/content/drive/MyDrive/Seed_learning_sub/{seed}_model_mlm_fold{i}.h5') for i in range(n_splits)]

#   # Load HLM models
#   hlm_models = [load_model(f'/content/drive/MyDrive/Seed_learning_sub/{seed}_model_hlm_fold{i}.h5') for i in range(n_splits)]

#   # Separate test features
#   x_test_mlm = test_input_MLM.drop(columns=['id', 'SMILES', 'MLM', 'HLM']).values
#   x_test_hlm = test_input_HLM.drop(columns=['id', 'SMILES', 'MLM', 'HLM']).values

#   mlm_predictions = []
#   hlm_predictions = []

#   for model_mlm, model_hlm in zip(mlm_models, hlm_models):
#       mlm_predictions.append(model_mlm.predict(x_test_mlm))
#       hlm_predictions.append(model_hlm.predict(x_test_hlm))

#   # Convert prediction lists to numpy arrays
#   mlm_predictions = np.array(mlm_predictions)
#   hlm_predictions = np.array(hlm_predictions)

#   mlm_ensemble_prediction = mlm_predictions.mean(axis=0)
#   hlm_ensemble_prediction = hlm_predictions.mean(axis=0)


#   df_submission = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/sample_submission.csv")
#   df_submission["MLM"] = mlm_ensemble_prediction
#   df_submission["HLM"] = hlm_ensemble_prediction
#   df_submission.to_csv(f"/content/drive/MyDrive/Seed_learning_sub/seed{seed}_submission.csv", index = False, encoding = "utf-8-sig")